## 1. Filtrando e tratando os dados

In [ ]:
# Vou refatorar esta parte. Enquanto isso, é necessário utilizar o arquivo filmes_tratado.csv

## 2. Adicionando novas Features

In [61]:
import pandas as pd
import numpy as np
import ast

df = pd.read_csv('filmes_rodrigo.csv')

### Tratamento

In [62]:
# Substituir orçamentos e receitas implausivelmente baixos por NaN
df['budget'] = df['budget'].replace(0, np.nan)
df['revenue'] = df['revenue'].replace(0, np.nan)

# Preencher os valores ausentes com a mediana 
df['budget'] = df['budget'].fillna(df['budget'].median())
df['revenue'] = df['revenue'].fillna(df['revenue'].median())

# Converter strings de listas/json em objetos 
def safe_literal_eval(val):
    try:
        if isinstance(val, str):
            return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return []
    return []

df['production_companies'] = df['production_companies'].apply(safe_literal_eval)
df['keywords'] = df['keywords'].apply(safe_literal_eval)

### Criando Features

In [ ]:
df['profit'] = df['revenue'] - df['budget']
df['roi'] = (df['profit']) / (df['budget'].replace(0, 1))
df['num_production_companies'] = df['production_companies'].apply(len)
df['num_keywords'] = df['keywords'].apply(len)
df['has_tagline'] = df['tagline'].notna().astype(int)
df['overview_length'] = df['overview'].astype(str).str.len()

### Seleção e Salvamento

In [ ]:
# Selecionar apenas as colunas que usaremos no modelo, incluindo a variável alvo
features_to_keep = [
    'popularity', 'budget', 'revenue', 'runtime', 'vote_count', 'vote_average',
    'release_year', 'release_month', 'release_day_of_week',
    'profit', 'roi', 'num_production_companies', 'num_keywords',
    'has_tagline', 'overview_length'
]
genre_cols = [col for col in df.columns if col.startswith('genre_')]
features_to_keep.extend(genre_cols)

# Criar o DataFrame final com apenas as colunas selecionadas
df_tratado = df[features_to_keep].copy()

# Lidar com possíveis valores infinitos e NaNs restantes
df_tratado.replace([np.inf, -np.inf], np.nan, inplace=True)
df_tratado.fillna(0, inplace=True)

output_filename = 'filmes_tratado.csv'
df_tratado.to_csv(output_filename, index=False)
print(f"\nTratamento concluído!")
print(f"O arquivo '{output_filename}' foi criado com sucesso e está pronto para ser usado no modelo Random Forest.")


Tratamento concluído!
O arquivo 'filmes_tratado.csv' foi criado com sucesso e está pronto para ser usado no modelo.
